In [10]:
import numpy as np
import pandas as pd
import scipy as sp
import re
import nltk
import string
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
df_train = pd.read_csv('./train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
df_train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [4]:
len(df_train['keyword'].unique())

222

In [5]:
df_train['location'].unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [6]:
len(df_train['location'].unique())

3342

In [7]:
df_train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [8]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
df_train.fillna('', inplace=True)

## Cleaning corpora

In [44]:
PUNCTUATION = set(string.punctuation)
STOP_WORDS = set(nltk.corpus.stopwords.words('english'))
lemm = nltk.WordNetLemmatizer()

In [45]:
def is_not_stopword(word):
    return word not in STOP_WORDS

def remove_punctuation(input):
    return ''.join([c for c in input if c not in PUNCTUATION])


def clean_text(input):
    tokens = re.split('\\s+', input.lower())
    tokens = filter(is_not_stopword, tokens)
    tokens = map(remove_punctuation, tokens)
    tokens = map(lemm.lemmatize, tokens)
    return ' '.join(tokens)


df_train['clean_text'] = df_train['text'].apply(clean_text)

df_train.head()

,id,keyword,location,text,target,clean_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,,,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,,,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [46]:
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=1000)
train_vectors = vectorizer.fit_transform(df_train['clean_text'])
train_vectors.shape

(7613, 1000)

In [50]:
model = linear_model.RidgeClassifierCV(cv=5, scoring='f1')
model.fit(train_vectors, df_train['target'])

RidgeClassifierCV(cv=5, scoring='f1')

In [51]:
model.best_score_

0.5905059104910763